for Bankruptsy dataset perform following algorithms
1] Logistic Regression
2] K-NN (1 to 16)
3] Gaussian NB(cross val score)
4] LDA
5] QDA

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,KFold,cross_val_score, GridSearchCV
import os
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('Bankruptcy.csv', index_col=0)
df.head()

,D,YR,R1,R2,R3,R4,R5,R6,R7,R8,...,R15,R16,R17,R18,R19,R20,R21,R22,R23,R24
NO,,,,,,,,,,,,,,,,,,,,,
1,0,78,0.23,0.08,0.02,0.03,0.46,0.12,0.19,10.36,...,0.05,0.57,0.15,0.23,3.56,0.26,1.55,0.43,0.11,0.17
2,0,77,0.19,0.07,0.09,0.12,0.02,0.02,0.03,3.13,...,0.09,0.12,0.16,0.22,3.78,1.29,1.40,0.06,0.07,0.10
3,0,72,0.07,0.02,0.03,0.05,0.06,0.10,0.14,2.41,...,-0.03,0.02,0.02,0.04,13.29,1.61,1.43,0.03,0.05,0.07
4,0,80,0.07,0.03,0.04,0.04,0.04,0.06,0.06,5.55,...,-0.02,0.01,0.02,0.02,5.36,1.30,1.12,-0.06,-0.08,-0.09
5,0,81,0.09,0.02,0.03,0.04,0.06,0.08,0.11,2.85,...,0.02,0.07,0.10,0.14,7.74,1.48,1.41,0.03,0.04,0.06


In [3]:
# checking the balancing
df['D'].value_counts()

D
0    66
1    66
Name: count, dtype: int64

In [4]:
X = df.drop(['D','YR'], axis=1)
y = df['D']

In [5]:
from sklearn.model_selection import StratifiedKFold  # IMPORTING the statifiedkfold insetd of only KFOLD 
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score,recall_score,f1_score
from sklearn.metrics import classification_report


### Logistic Regression

In [6]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold 

params = {
    'penalty': ['l2',None],'C':np.linspace(0.001,10,20),  # Try both L1 (Lasso) and L2 (Ridge) penalties
    'solver': ['lbfgs', 'sag','newton-cg', 'newton-cholesky']  # Different solvers
}

lr = LogisticRegression()
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=24)
gcv_r= GridSearchCV(lr,param_grid=params ,scoring='neg_log_loss',cv=kfold)
gcv_r.fit(X, y)

print(gcv_r.best_params_)
print(gcv_r.best_score_)

{'C': 0.5272631578947369, 'penalty': 'l2', 'solver': 'lbfgs'}
-0.4658204056787668


### KNN Classifier

In [7]:
from  sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score,classification_report
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

mm_scaler=MinMaxScaler()
std_scaler=StandardScaler()
knn=KNeighborsClassifier()

params={"KNN__n_neighbors":np.arange(1,16),"SCL":[None,std_scaler,mm_scaler]}

pipe=Pipeline([("SCL",None),("KNN",knn)])

kfold=StratifiedKFold(n_splits=5,shuffle=True,random_state=24)

gcv=GridSearchCV(pipe,param_grid=params,cv=kfold,scoring="neg_log_loss")  # verbose 1/2/3 => gives step by step execution
gcv.fit(X, y)

print(gcv.best_params_)
print(gcv.best_score_)

{'KNN__n_neighbors': 15, 'SCL': StandardScaler()}
-0.45577248787788677


### Guassian NB 

In [8]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold


kfold = StratifiedKFold(n_splits=5, shuffle= True, random_state=24)
nb = GaussianNB()
results = cross_val_score(nb, X, y, cv=kfold)
print("Accuracy score : ",results.mean())

# with log loss 
results = cross_val_score(nb, X, y, cv=kfold, scoring='neg_log_loss')
print('Log loss : ',results.mean())


Accuracy score :  0.7495726495726496
Log loss :  -2.5042421629200953


###  LDA and QDA

In [9]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()
qda = QuadraticDiscriminantAnalysis()

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=24)
res_lda = cross_val_score(lda, X, y, cv = kfold,scoring='neg_log_loss')
print("LDA cross log loss score : ",res_lda.mean())

res_qda = cross_val_score(qda, X,y, cv = kfold,scoring='neg_log_loss')
print('QDA cross log loss score : ', res_qda.mean())

LDA cross log loss score :  -0.7280259190109073
QDA cross log loss score :  -5.455942905787956


### Log loss for diffrent Models 

#### Logistic Regression =>  -0.4659701148359128
#### KNN Classifier          =>  -0.45577248787788677
#### Guassian NB             =>  -2.5042421629200953
####  LDA                           =>  -0.72802591901091
####  QDA                           =>  -5.455942905787905

### Best Model for this dataset =>>  KNN Classifier with log_loss -0.45577248787788677


In [10]:
X.columns

Index(['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R7', 'R8', 'R9', 'R10', 'R11',
       'R12', 'R13', 'R14', 'R15', 'R16', 'R17', 'R18', 'R19', 'R20', 'R21',
       'R22', 'R23', 'R24'],
      dtype='object')

In [11]:
### to Enable log loss
from sklearn.svm import SVC

mm_scaler=MinMaxScaler()
std_scaler=StandardScaler()
svc = SVC(random_state=24,probability=True)

params = {'SVC__kernel':['rbf','linear'],
          'SVC__C':np.linspace(0.01,4,10),
         'SVC__gamma':np.linspace(0.01,4,10),
          "SCL":[None,std_scaler,mm_scaler]}

pipe=Pipeline([("SCL",None),("SVC",svc)])

kfold=StratifiedKFold(n_splits=5,shuffle=True,random_state=24)

gcv=GridSearchCV(pipe,param_grid=params,cv=kfold,scoring='neg_log_loss', verbose=1)  # verbose 1/2/3 => gives step by step execution
gcv.fit(X,y)

print(gcv.best_params_)
print(gcv.best_score_)

Fitting 5 folds for each of 600 candidates, totalling 3000 fits
{'SCL': StandardScaler(), 'SVC__C': 0.45333333333333337, 'SVC__gamma': 0.01, 'SVC__kernel': 'rbf'}
-0.46457037675584906
